# Validation Playground

**Watch** a [short tutorial video](https://greatexpectations.io/videos/getting_started/integrate_expectations) or **read** [the written tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data)

#### This notebook assumes that you created at least one expectation suite in your project.
#### Here you will learn how to validate data in a SQL database against an expectation suite.


We'd love it if you **reach out for help on** the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack)

In [1]:
import json
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.datasource.types import BatchKwargs
import datetime

2020-07-30T00:51:28-0400 - INFO - Great Expectations logging enabled at 20 level by JupyterUX module.


## 1. Get a DataContext
This represents your **project** that you just created using `great_expectations init`.

In [2]:
context = ge.data_context.DataContext()

## 2. Choose an Expectation Suite

List expectation suites that you created in your project

In [3]:
context.list_expectation_suite_names()

['taxi.demo']

In [4]:
expectation_suite_name =  'taxi.demo'

## 3. Load a batch of data you want to validate

To learn more about `get_batch`, see [this tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data#load-a-batch-of-data-to-validate)


In [5]:
# list datasources of the type SqlAlchemyDatasource in your project
[datasource['name'] for datasource in context.list_datasources() if datasource['class_name'] == 'SqlAlchemyDatasource']

['my_postgres_db']

In [6]:
datasource_name = 'my_postgres_db'

In [7]:
# If you would like to validate an entire table or view in your database's default schema:
batch_kwargs = {'table': "yellow_tripdata_staging", 'datasource': datasource_name}

# # If you would like to validate an entire table or view from a non-default schema in your database:
# batch_kwargs = {'table': "YOUR_TABLE", "schema": "YOUR_SCHEMA", 'datasource': datasource_name}

# If you would like to validate the result set of a query:
# batch_kwargs = {'query': 'SELECT YOUR_ROWS FROM YOUR_TABLE', 'datasource': datasource_name}



batch = context.get_batch(batch_kwargs, expectation_suite_name)
batch.head()

2020-07-30T00:52:27-0400 - INFO - 	8 expectation(s) included in expectation_suite.


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-02-07 15:48:06,2019-02-07 16:00:40,2.0,1.30,1,N,234.0,211.0,1,9.0,2.5,0.5,2.46,0.0,0.3,14.76,2.5
1,2,2019-02-11 15:19:56,2019-02-11 15:40:10,1.0,2.37,1,N,161.0,249.0,1,14.0,0.0,0.5,3.46,0.0,0.3,20.76,2.5
2,2,2019-02-15 20:03:53,2019-02-15 20:08:34,1.0,0.48,1,N,237.0,161.0,1,5.0,0.5,0.5,1.50,0.0,0.3,10.30,2.5
3,2,2019-02-03 15:16:04,2019-02-03 15:21:49,1.0,0.64,1,N,100.0,48.0,1,5.5,0.0,0.5,1.76,0.0,0.3,10.56,2.5
4,2,2019-02-15 09:23:09,2019-02-15 09:42:51,6.0,2.56,1,N,114.0,230.0,1,13.5,0.0,0.5,3.36,0.0,0.3,20.16,2.5


## 4. Validate the batch with Validation Operators

`Validation Operators` provide a convenient way to bundle the validation of
multiple expectation suites and the actions that should be taken after validation.

When deploying Great Expectations in a **real data pipeline, you will typically discover these needs**:

* validating a group of batches that are logically related
* validating a batch against several expectation suites such as using a tiered pattern like `warning` and `failure`
* doing something with the validation results (e.g., saving them for a later review, sending notifications in case of failures, etc.).

[Read more about Validation Operators in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data#save-validation-results)

In [8]:
# This is an example of invoking a validation operator that is configured by default in the great_expectations.yml file

"""
Create a run_id. The run_id must be of type RunIdentifier, with optional run_name and run_time instantiation
arguments (or a dictionary with these keys). The run_name can be any string (this could come from your pipeline
runner, e.g. Airflow run id). The run_time can be either a dateutil parsable string or a datetime object.
Note - any provided datetime will be assumed to be a UTC time. If no instantiation arguments are given, run_name will
be None and run_time will default to the current UTC datetime.
"""

run_id = {
  "run_name": "validation_run",  # insert your own run_name here
  "run_time": datetime.datetime.now(datetime.timezone.utc)
}

results = context.run_validation_operator(
    "action_list_operator",
    assets_to_validate=[batch],
    run_id=run_id)

2020-07-30T00:52:46-0400 - INFO - 	8 expectation(s) included in expectation_suite.
2020-07-30T00:52:48-0400 - WARNING - Using lossy conversion for decimal 12.6200800000000000 to float object to support serialization.


## 5. View the Validation Results in Data Docs

Let's now build and look at your Data Docs. These will now include an **data quality report** built from the `ValidationResults` you just created that helps you communicate about your data with both machines and humans.

[Read more about Data Docs in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/validate_data.html?utm_source=notebook&utm_medium=validate_data#view-the-validation-results-in-data-docs)

In [9]:
context.open_data_docs()

## Congratulations! You ran Validations!

## Next steps:

### 1. Read about the typical workflow with Great Expectations:

[typical workflow](https://docs.greatexpectations.io/en/latest/getting_started/typical_workflow.html?utm_source=notebook&utm_medium=validate_data#view-the-validation-results-in-data-docs)

### 2. Explore the documentation & community

You are now among the elite data professionals who know how to build robust descriptions of your data and protections for pipelines and machine learning models. Join the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack) to see how others are wielding these superpowers.